In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
# import requests
import io
import zipfile
import xmltodict
from io import BytesIO
from zipfile import ZipFile
from xml.etree.ElementTree import parse
# from bs4 import BeautifulSoup
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

## 고유번호 개발 가이드
1. 요청 파라미터
- 고유번호 corp_code 
- 정식명칭 corp_name
- 종목코드 stock_code
- 최종변경일자 modify_date

2. 추출 결과값
- 

In [5]:
# https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019018
KEY = '5b8f0aa6967605a94ba6e9a39b461c2e86cf0d85'

url = '	https://opendart.fss.or.kr/api/corpCode.xml'
params = {'crtfc_key' : KEY}

response = requests.get(url, params = params).content


# 압축 파일 풀어서 xml파일 디렉토리에 저장하기
with ZipFile(BytesIO(response)) as zipfile:
    zipfile.extractall('corpCode')

#corpcode xml 파일 생김

In [11]:
# xml 파일 읽어오기, 여기서 고유번호와 종목명이 포함된 리스트를 만들 것임
xmlTree = parse('./corpCode/CORPCODE.xml')
root = xmlTree.getroot()
raw_list = root.findall('list')

corp_list = []

for l in range(len(raw_list)):
    corp_code = raw_list[l].findtext('corp_code')
    corp_name = raw_list[l].findtext('corp_name')
    stock_code = raw_list[l].findtext('stock_code')
    modify_date = raw_list[l].findtext('modify_date')
    
    corp_list.append([corp_code, corp_name, stock_code, modify_date])
    
#corp_list 형성

In [21]:
corp_list

[['00434003', '다코', ' ', '20170630'],
 ['00434456', '일산약품', ' ', '20170630'],
 ['00430964', '굿앤엘에스', ' ', '20170630'],
 ['00432403', '한라판지', ' ', '20170630'],
 ['00388953', '크레디피아제이십오차유동화전문회사', ' ', '20170630'],
 ['00179984', '연방건설산업', ' ', '20170630'],
 ['00420143', '브룩스피알아이오토메이션잉크', ' ', '20170630'],
 ['00401111', '매경아이비아이', ' ', '20170630'],
 ['00435534', '캐드뱅크', ' ', '20170630'],
 ['00430186', '엠와이오피삼차유동화전문유한회사', ' ', '20170630'],
 ['00430201', '엠와이오피이차유동화전문유한회사', ' ', '20170630'],
 ['00430210', '엠와이오피일차유동화전문유한회사', ' ', '20170630'],
 ['00430229', '포스미디어', ' ', '20170630'],
 ['00140432', '축복할렐루야', ' ', '20170630'],
 ['00426208', '한국전자화학', ' ', '20170630'],
 ['00433262', 'ConnachtCapitalMarketInvestmentLtd.', ' ', '20170630'],
 ['00433749', '선진아이티', ' ', '20170630'],
 ['00433785', '팀스코리아', ' ', '20170630'],
 ['00196079', '에넥스하이테크', ' ', '20170630'],
 ['00435048', '세이스텝바이스텝혼합형펀드', ' ', '20170630'],
 ['00435057', '유리알파헷지채권혼합형펀드', ' ', '20170630'],
 ['00108843', '한기술정보통신', ' ', '2017063

In [12]:
#corp_list -> corp_df


from pandas import DataFrame
from datetime import datetime

#위에서 생성한 corp_list에 라벨링하여 데이터프레임화

corp_df = DataFrame(corp_list, columns=[
    '고유번호', 
    '정식명칭', 
    '종목코드', 
    '최종변경일자'
])

#corp_df 의 결측치 drop시킨 stock_df (종목코드가 없는 종목을 drop하고, 최종변경일자는 쓰지 않으므로 칼럼 전체를 drop)

stock_df = corp_df[corp_df['종목코드'] != " "]
stock_df = stock_df[['고유번호', '정식명칭', '종목코드']].dropna()
stock_df

,고유번호,정식명칭,종목코드
2008,00260985,한빛네트,036720
2020,00264529,엔플렉스,040130
2021,00358545,동서정보기술,055000
2783,00231567,애드모바일,032600
3886,00247939,씨모스,037600
...,...,...,...
101145,00296078,APS,054620
101146,01118643,로보쓰리에이아이앤로보틱스,238500
101147,00624518,다산솔루에타,154040
101148,01065679,알비케이그룹,215790


In [63]:
# stock_df에서 고유번호를 호출해오는 함수 형성
def goyu_searcher(COMPANYNAME):
    temp_df = stock_df.loc[(stock_df['정식명칭'] == COMPANYNAME)]
    return temp_df.iloc[0, 0]

In [76]:
# 가져온 corp_code를 바탕으로 

def get_items(KEY, CORP_CODE, YEAR, RPT_CODE):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    from lxml import html
    from urllib.request import Request, urlopen
    from urllib.parse import urlencode, quote_plus, unquote
    
    
    url = 'https://opendart.fss.or.kr/api/fnlttMultiAcnt.json'
    parmas={'crtfc_key' : KEY,
            'corp_code' : CORP_CODE, 
            'bsns_year' : YEAR, 
            'reprt_code' : RPT_CODE}
    
    response = requests.get(url, params = params)
    
    import json
    json_obj = json.loads(response.content.decode('UTF-8'))
    
    return json_obj

In [77]:
get_items(KEY, '00126380', '2018', '11011')

{'status': '100', 'message': '필수값(corp_code,bsns_year,reprt_code)이 누락되었습니다.'}

In [ ]:
#get_items()를 사용하여 구하고자 하는 기업들의 재무제표 정보 가져오기

corp_list = ['삼성전자', '삼성생명', '현대자동차', '셀트리온', 'POSCO홀딩스']
year_list = range(2018, 2024)
reprtlist = ['11013', '11012', '11014', '11011']

for corp in corp_list:
    for year in year_list:
        for reprt in reprtlist:
            get_items(KEY, goyu_searcher(corp), year, reprt)
            

In [71]:
goyu_searcher('POSCO홀딩스')

'00155319'

## OpenDartReader 클래스 사용

In [66]:
import OpenDartReader
import pandas as pd

dart = OpenDartReader(KEY)
# dfdf = dart.finstate('005930, 032830, 005380, 068270, 005490', 2022)


corp_list = ['삼성전자', '삼성생명', '현대자동차', '셀트리온', 'POSCO홀딩스']
year_list = range(2018, 2024)
reprtlist = ['11013', '11012', '11014', '11011']


dfs = []
for corp in corp_list:
    for year in year_list:
        for report in reprtlist:
            singledf = dart.finstate(corp, year, reprt_code = report)
            dfs.append(singledf)



result = pd.concat(dfs, ignore_index=True)
result




{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,...,frmtrm_nm,frmtrm_dt,frmtrm_amount,ord,currency,thstrm_add_amount,frmtrm_add_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount
0,20180515001699,11013,2018,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동자산,...,제 49 기말,2017.12.31 현재,"146,982,464,000,000",1,KRW,NaN,NaN,NaN,NaN,NaN
1,20180515001699,11013,2018,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동자산,...,제 49 기말,2017.12.31 현재,"154,769,626,000,000",3,KRW,NaN,NaN,NaN,NaN,NaN
2,20180515001699,11013,2018,00126380,005930,CFS,연결재무제표,BS,재무상태표,자산총계,...,제 49 기말,2017.12.31 현재,"301,752,090,000,000",5,KRW,NaN,NaN,NaN,NaN,NaN
3,20180515001699,11013,2018,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동부채,...,제 49 기말,2017.12.31 현재,"67,175,114,000,000",7,KRW,NaN,NaN,NaN,NaN,NaN
4,20180515001699,11013,2018,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동부채,...,제 49 기말,2017.12.31 현재,"20,085,548,000,000",9,KRW,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1593,20230515001841,11013,2023,00155319,005490,OFS,재무제표,BS,재무상태표,이익잉여금,...,제 55 기말,2022.12.31 현재,"47,409,674,639,879",19,KRW,NaN,NaN,NaN,NaN,NaN
1594,20230515001841,11013,2023,00155319,005490,OFS,재무제표,BS,재무상태표,자본총계,...,제 55 기말,2022.12.31 현재,"47,171,862,677,677",22,KRW,NaN,NaN,NaN,NaN,NaN
1595,20230515001841,11013,2023,00155319,005490,OFS,재무제표,IS,손익계산서,영업이익,...,제 55 기1분기,2022.01.01 ~ 2022.03.31,"1,069,678,186,525",26,KRW,"493,890,425,693","1,069,678,186,525",NaN,NaN,NaN
1596,20230515001841,11013,2023,00155319,005490,OFS,재무제표,IS,손익계산서,법인세차감전 순이익,...,제 55 기1분기,2022.01.01 ~ 2022.03.31,"1,078,155,662,943",28,KRW,"513,147,642,359","1,078,155,662,943",NaN,NaN,NaN


In [67]:
result = result.drop(columns=['ord', 'rcept_no', 'stock_code'])
result

,reprt_code,bsns_year,corp_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,currency,thstrm_add_amount,frmtrm_add_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount
0,11013,2018,00126380,CFS,연결재무제표,BS,재무상태표,유동자산,제 50 기1분기말,2018.03.31 현재,"154,941,953,000,000",제 49 기말,2017.12.31 현재,"146,982,464,000,000",KRW,NaN,NaN,NaN,NaN,NaN
1,11013,2018,00126380,CFS,연결재무제표,BS,재무상태표,비유동자산,제 50 기1분기말,2018.03.31 현재,"157,531,160,000,000",제 49 기말,2017.12.31 현재,"154,769,626,000,000",KRW,NaN,NaN,NaN,NaN,NaN
2,11013,2018,00126380,CFS,연결재무제표,BS,재무상태표,자산총계,제 50 기1분기말,2018.03.31 현재,"312,473,113,000,000",제 49 기말,2017.12.31 현재,"301,752,090,000,000",KRW,NaN,NaN,NaN,NaN,NaN
3,11013,2018,00126380,CFS,연결재무제표,BS,재무상태표,유동부채,제 50 기1분기말,2018.03.31 현재,"68,298,586,000,000",제 49 기말,2017.12.31 현재,"67,175,114,000,000",KRW,NaN,NaN,NaN,NaN,NaN
4,11013,2018,00126380,CFS,연결재무제표,BS,재무상태표,비유동부채,제 50 기1분기말,2018.03.31 현재,"20,914,647,000,000",제 49 기말,2017.12.31 현재,"20,085,548,000,000",KRW,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1593,11013,2023,00155319,OFS,재무제표,BS,재무상태표,이익잉여금,제 56 기1분기말,2023.03.31 현재,"47,809,907,948,646",제 55 기말,2022.12.31 현재,"47,409,674,639,879",KRW,NaN,NaN,NaN,NaN,NaN
1594,11013,2023,00155319,OFS,재무제표,BS,재무상태표,자본총계,제 56 기1분기말,2023.03.31 현재,"47,696,772,703,843",제 55 기말,2022.12.31 현재,"47,171,862,677,677",KRW,NaN,NaN,NaN,NaN,NaN
1595,11013,2023,00155319,OFS,재무제표,IS,손익계산서,영업이익,제 56 기1분기,2023.01.01 ~ 2023.03.31,"493,890,425,693",제 55 기1분기,2022.01.01 ~ 2022.03.31,"1,069,678,186,525",KRW,"493,890,425,693","1,069,678,186,525",NaN,NaN,NaN
1596,11013,2023,00155319,OFS,재무제표,IS,손익계산서,법인세차감전 순이익,제 56 기1분기,2023.01.01 ~ 2023.03.31,"513,147,642,359",제 55 기1분기,2022.01.01 ~ 2022.03.31,"1,078,155,662,943",KRW,"513,147,642,359","1,078,155,662,943",NaN,NaN,NaN


In [88]:


dfdfdf = pd.pivot_table(result, 
                        index = ['corp_code', 'bsns_year', 'reprt_code', 'thstrm_dt', 'account_nm', 'thstrm_amount'])

pd.set_option('display.max_rows', None)
dfdfdf

/Users/mj/opt/anaconda3/lib/python3.9/site-packages/pandas/core/apply.py:507: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  return self._try_aggregate_string_function(obj, f, *self.args, **self.kwargs)


Empty DataFrame
Columns: []
Index: [(00126380, 2018, 11011, 2018.01.01 ~ 2018.12.31, 당기순이익, 32,815,127,000,000), (00126380, 2018, 11011, 2018.01.01 ~ 2018.12.31, 당기순이익, 44,344,857,000,000), (00126380, 2018, 11011, 2018.01.01 ~ 2018.12.31, 매출액, 170,381,870,000,000), (00126380, 2018, 11011, 2018.01.01 ~ 2018.12.31, 매출액, 243,771,415,000,000), (00126380, 2018, 11011, 2018.01.01 ~ 2018.12.31, 법인세차감전 순이익, 44,398,855,000,000), (00126380, 2018, 11011, 2018.01.01 ~ 2018.12.31, 법인세차감전 순이익, 61,159,958,000,000), (00126380, 2018, 11011, 2018.01.01 ~ 2018.12.31, 영업이익, 43,699,451,000,000), (00126380, 2018, 11011, 2018.01.01 ~ 2018.12.31, 영업이익, 58,886,669,000,000), (00126380, 2018, 11011, 2018.12.31 현재, 부채총계, 46,033,232,000,000), (00126380, 2018, 11011, 2018.12.31 현재, 부채총계, 91,604,067,000,000), (00126380, 2018, 11011, 2018.12.31 현재, 비유동부채, 2,888,179,000,000), (00126380, 2018, 11011, 2018.12.31 현재, 비유동부채, 22,522,557,000,000), (00126380, 2018, 11011, 2018.12.31 현재, 비유동자산, 138,981,902,000,000), (00126380, 2018, 11011, 2018.12.31 현재, 비유동자산, 164,659,820,000,000), (00126380, 2018, 11011, 2018.12.31 현재, 유동부채, 43,145,053,000,000), (00126380, 2018, 11011, 2018.12.31 현재, 유동부채, 69,081,510,000,000), (00126380, 2018, 11011, 2018.12.31 현재, 유동자산, 174,697,424,000,000), (00126380, 2018, 11011, 2018.12.31 현재, 유동자산, 80,039,455,000,000), (00126380, 2018, 11011, 2018.12.31 현재, 이익잉여금, 166,555,532,000,000), (00126380, 2018, 11011, 2018.12.31 현재, 이익잉여금, 242,698,956,000,000), (00126380, 2018, 11011, 2018.12.31 현재, 자본금, 897,514,000,000), (00126380, 2018, 11011, 2018.12.31 현재, 자본총계, 172,988,125,000,000), (00126380, 2018, 11011, 2018.12.31 현재, 자본총계, 247,753,177,000,000), (00126380, 2018, 11011, 2018.12.31 현재, 자산총계, 219,021,357,000,000), (00126380, 2018, 11011, 2018.12.31 현재, 자산총계, 339,357,244,000,000), (00126380, 2018, 11012, 2018.01.01 ~ 2018.06.30, 당기순이익, 11,043,412,000,000), (00126380, 2018, 11012, 2018.01.01 ~ 2018.06.30, 당기순이익, 8,077,985,000,000), (00126380, 2018, 11012, 2018.01.01 ~ 2018.06.30, 매출액, 41,314,723,000,000), (00126380, 2018, 11012, 2018.01.01 ~ 2018.06.30, 매출액, 58,482,658,000,000), (00126380, 2018, 11012, 2018.01.01 ~ 2018.06.30, 법인세차감전 순이익, 11,094,590,000,000), (00126380, 2018, 11012, 2018.01.01 ~ 2018.06.30, 법인세차감전 순이익, 15,406,836,000,000), (00126380, 2018, 11012, 2018.01.01 ~ 2018.06.30, 영업이익, 11,049,605,000,000), (00126380, 2018, 11012, 2018.01.01 ~ 2018.06.30, 영업이익, 14,869,035,000,000), (00126380, 2018, 11012, 2018.06.30 현재, 부채총계, 42,130,475,000,000), (00126380, 2018, 11012, 2018.06.30 현재, 부채총계, 85,563,535,000,000), (00126380, 2018, 11012, 2018.06.30 현재, 비유동부채, 1,926,719,000,000), (00126380, 2018, 11012, 2018.06.30 현재, 비유동부채, 19,961,212,000,000), (00126380, 2018, 11012, 2018.06.30 현재, 비유동자산, 135,899,888,000,000), (00126380, 2018, 11012, 2018.06.30 현재, 비유동자산, 161,711,541,000,000), (00126380, 2018, 11012, 2018.06.30 현재, 유동부채, 40,203,756,000,000), (00126380, 2018, 11012, 2018.06.30 현재, 유동부채, 65,602,323,000,000), (00126380, 2018, 11012, 2018.06.30 현재, 유동자산, 156,976,839,000,000), (00126380, 2018, 11012, 2018.06.30 현재, 유동자산, 68,070,224,000,000), (00126380, 2018, 11012, 2018.06.30 현재, 이익잉여금, 159,960,751,000,000), (00126380, 2018, 11012, 2018.06.30 현재, 이익잉여금, 231,065,173,000,000), (00126380, 2018, 11012, 2018.06.30 현재, 자본금, 897,514,000,000), (00126380, 2018, 11012, 2018.06.30 현재, 자본총계, 161,839,637,000,000), (00126380, 2018, 11012, 2018.06.30 현재, 자본총계, 233,124,845,000,000), (00126380, 2018, 11012, 2018.06.30 현재, 자산총계, 203,970,112,000,000), (00126380, 2018, 11012, 2018.06.30 현재, 자산총계, 318,688,380,000,000), (00126380, 2018, 11013, 2018.01.01 ~ 2018.03.31, 당기순이익, 11,688,544,000,000), (00126380, 2018, 11013, 2018.01.01 ~ 2018.03.31, 당기순이익, 8,452,458,000,000), (00126380, 2018, 11013, 2018.01.01 ~ 2018.03.31, 매출액, 42,606,978,000,000), (00126380, 2018, 11013, 2018.01.01 ~ 2018.03.31, 매출액, 60,563,714,000,000), (00126380, 2018, 11013, 2018.01.01 ~ 2018.03.31, 법인세차감전 순이익, 11,568,977,000,000), (00126380, 2018, 11013, 2018.01.01 ~ 2018.03.31, 법인세차감전 순이익, 16,175,907,00

In [89]:
dfdfdf.to_csv('./apiresult.csv', encoding='euc-kr')

In [38]:
KEY = '5b8f0aa6967605a94ba6e9a39b461c2e86cf0d85'
YEAR = '2021'
RPT_CODE = '11011' 
# 1분기보고서 : 11013
# 반기보고서 : 11012
# 3분기보고서 : 11014
# 사업보고서 : 11011

# 가져오고자 하는 기업의 이름이 담긴 리스트가 있을 거야
corp_list = ['삼성전자', '삼성생명', '현대자동차', '셀트리온', 'POSCO홀딩스']
year_list = range(2018, 2024)

item_list =[
    'bsns_year',
    'stock_code',
    'reprt_code',
    'fs_div',
    'sj_div',
    'account_nm',
    'thstrm_nm',
    'thstrm_dt',
    'thstrm_amount',
    'thstrm_add_amount',
    'frmtrm_nm',
    'frmtrm_dt',
    'frmtrm_amount',
    'frmtrm_add_amount',
    'bfefrmtrm_nm',
    'bfefrmtrm_dt',
    'bfefrmtrm_amount',
    'currency'
]

fs_list = []
for corp in corp_list:
    CORP_CODE = goyu_searcher(corp)
    
    items = get_items(KEY, CORP_CODE, YEAR, RPT_CODE)
    
    for i in range(0, len(items)):
        fs_item_list = []
        for item in item_list:
            try:
                value = items[i].find(item).text
            except:
                value = ''
            fs_item_list.append(value)
            
        fs_list.append(fs_item_list)
        
    print(CORP_CODE)


005930
032830
005380
068270
005490


## dart-fss

In [37]:
!pip install dart-fss

     |████████████████████████████████| 141 kB 10.5 MB/s eta 0:00:01
     |████████████████████████████████| 8.0 MB 11.1 MB/s eta 0:00:01
  Using cached fake_useragent-1.1.3-py3-none-any.whl (50 kB)
  Using cached importlib_resources-5.12.0-py3-none-any.whl (36 kB)
     |████████████████████████████████| 41 kB 3.2 MB/s eta 0:00:01
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
  Created wheel for halo: filename=halo-0.0.31-py3-none-any.whl size=11260 sha256=7e1539c7dda11c144fa3a78694581a10de3c2fbada724de8085d932753419184
  Stored in directory: /Users/mj/Library/Caches/pip/wheels/bb/85/47/b7c7338ab52808105f937bd8c04aec5d98a543311ac2c8bed2
Successfully built halo


In [44]:
import dart_fss as dart

KEY = '5b8f0aa6967605a94ba6e9a39b461c2e86cf0d85'
dart.set_api_key(api_key = KEY)

corp_list = dart.get_corp_list()



AttributeError: 'list' object has no attribute 'info'

In [46]:
len(list(corp_list))

101136

In [48]:
# 삼성전자 검색
samsung = corp_list.find_by_corp_name('삼성전자', exactly=True)[0]

# 2012년부터 연간 연결재무제표 불러오기
fs = samsung.extract_fs(bgn_de='20160101')

# 재무제표 검색 결과를 엑셀파일로 저장 ( 기본저장위치: 실행폴더/fsdata )
fs.save()

Annual reports:   0%|          | 0/8 [00:00<?, ?report/s]

'/Users/mj/Desktop/workspace/final_pjt/fsdata/00126380_annual.xlsx'

## 테스트